# Built-in test runner

## File and folder conventions

The Mojo CLI has a built-in test runner that can be used to test Mojo programs. `mojo test` will recursively search the `test` directory for any files with `test` in the filename. So in the following example:
```bash
ls ./test
test_users_controller.mojo
factories.mojo
./test/models:
user_model_tests.mojo
```
... the `mojo` CLI will search `test_users_controller.mojo` and `user_model_tests.mojo` for tests, but not `factories.mojo`. 

## Test functions

Test functions must be named `test_*` and take no arguments. Use `def` notation rather than `fn` notation. If you use `fn` notation then, in the presence of any assertion  functions, the compiler will complain unless you wrap the body in a `try/exception` block. So:
```mojo
def test_foo():
    assert_equal(1, 1)
```

Not:
```mojo
fn test_foo():
    assert_equal(1, 1)
```

